In [ ]:
from __future__ import division
import pylab as plt
from numpy import random,argwhere,argmax,linspace,logspace,var,mean,tile,dot
import copy
from numpy.random import randint
from scipy.stats import norm
from scipy import sparse
import numpy as np
%matplotlib inline
from matplotlib import pyplot
import warnings
warnings.filterwarnings('ignore')
import scipy.io as sio
from collections import Counter
import matplotlib as mpl
from scipy.stats import nbinom
import os
import pickle

In [ ]:
import json

In [ ]:
from morphopy.computation import file_manager as fm
from morphopy.neurontree import NeuronTree as nt
import pandas as pd
import networkx as nx
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
import cma

In [ ]:
import modmorf
import modsim
import modplot

In [ ]:
script_dir = ## os.path.dirname(__file__) 

In [ ]:
save_dir = ## os.path.dirname(__file__) 

## Load morphology 

In [ ]:
morphofile = "nobranchtree.swc"
Nt = fm.load_swc_file('//morphologies//'+morphofile)

In [ ]:
Axon = Nt.get_axonal_tree().resample_tree(dist=0.1) # get axonal tree only (still includes the soma point)  # resample the neuron tree, d=1um 
DirA = Axon.get_adjacency_matrix(weight=None) # adjacency matrix of resampled axonal tree (directed matrix - only children connections)
A = DirA + np.transpose(DirA)

In [ ]:
N = modmorf.get_number_of_nodes(A) # number of nodes/positions
print(N)
# can resample tree with smaller distance if want to increase numebr of nodes to simulate on for parameter optimisation

In [ ]:
#Parent and Children only matrices
#Parent, Children, ChildrenWeighed = relations_nodes(A, N)  # Parent, Children (edges = 1), ChildrenWeighed (edges = 0.5 if following BP, else =1)
Parent, Children = modmorf.relations_nodes(A)  # Parent, Children (edges = 1)

In [ ]:
##Rename nodes to match adjacency matrix indexes - the new naming of nodes will be used in all following dicitonaries called

# first, create a dictionary of indices 'mydict' {old ind: new ind} to pass as 'label' argument in rename function

mydict = Axon.get_node_attributes('type') # using any of the functions to bring up a dictionary with current indexing system
for i, key in enumerate(mydict):  # iterates on the keys (ie current indices)  
    mydict[key] = i               # give value 0 to n

Axon.rename_nodes_zerostart(label=mydict) # rename nodes to match adj matrix indexes 

In [ ]:
BP = Axon.get_branchpoints() # branchpoints array
EP = Axon.get_tips() # endpoints array

# children of branchpoints array
CBP = []   
for i in range(len(BP)):
    CBP.append((Children[BP[i],:]).indices)
CBP = np.array(CBP)   

NodeType = Axon.get_node_attributes('type') # type of node dictionary - 1 is soma, 2 is axon 
BO = Axon.get_branch_order() # branch order dictionary 
BOposN = Counter(BO.values()) # number of nodes per BOid
maxBO = len(BOposN) # highest branch order 

In [ ]:
ChildrenWeighed = modmorf.child_matrix_weighed(Children, N, BP)  # ChildrenWeighed (edges = 0.5 if following BP, else =1)

## Simulation Parameters 

In [ ]:
#states: off = 0, pause = 1, retrograde = 2, anterograde = 3 

#select genotype below by commenting/uncommenting
genotype = 'wt' 
#genotype = 'tau'

timestep = 1.35

numtimesteps = 223+100 # number of timsteps per particle (mitochondrion) run
runs = 20000  #number of mitochondria 


randompos = False
randomstate = True
x0 = int(N/2) if randompos == False else None # x0 is None if randompos is True
s0 = 1 if randomstate == False else None

#states: off = 0, pause = 1, retrograde = 2, anterograde = 3 
statesarray = np.array([0,1,2,3]) 

    #if randomstate = True, choosing steadyinit=False means that steady state (eigenvectors) will be used instead of specified states start probabilities below
steadyinit = True 
states_startprob = [0.25, 0.25, 0.25, 0.25] if steadyinit == False else None 

adjusted_velocity = 1.0 # not relevant for optimisation but part of simulation code so needs to be given a value


In [ ]:
paramfixed  = {'genotype':genotype, 'numtimesteps': numtimesteps, 'runs': runs, 'statesarray':statesarray, 'adjusted_velocity':adjusted_velocity, 
               'N': N, 'x0':x0, 's0': s0, 'states_startprob':states_startprob, 'randompos':randompos, 'randomstate':randomstate, 'steadyinit':steadyinit,
              'EP':EP, 'BP':BP, 'CBP':CBP, 'morphofile': morphofile}

## Parameter-arrays building functions (for optimisation)

In [ ]:
def symmetrical_parameters(param_seven, scale=False): 
    # does the scaling to 1 need to happen before or after symmetry?? I think either gives same result
    
    """
    returns 
        fourteen parameter array converted from seven parameter array by symmetry 
        ie splitting 'moving' state to 'anterograde' and 'retrograde' states
    
    args
        param_seven = array of seven parameters, if obtained from unconstrinaed optimisation, needs to be scaled
        scale = (False by defualt)/ pass True if used on param_seven from unconstrained optimsation  so that probabilities to add up to 1
    
    """
    
    #STEP1 -if scale == True 
    #scale the seven parameters (returned from optimisation) to have probabilities of transitionng from a state add up to 1
    
    if scale == True:
    
        param_seven_scaled = []

        sum1 = param_seven[0]+param_seven[1]
        sum2 = param_seven[2]+param_seven[3]
        sum3 = param_seven[4]+param_seven[5]+param_seven[6]

        for i in range(2):
            param_seven_scaled.append(param_seven[i]/sum1)
        for i in range(2,4):
            param_seven_scaled.append(param_seven[i]/sum2) 
        for i in range(4,7):
            param_seven_scaled.append(param_seven[i]/sum3)
    
    else:
        param_seven_scaled = np.copy(param_seven)
    
    #STEP2
    #convert 7 transitions parameters (pause, off-track, moving) into full (14) parameters transitions
    
    #param_seven = [s0s0, s0s3(*2), s1s1, s1s3(*2), s3s0, s3s1, s3s3(*2)]
    #param_seven = [s0s0, s0sM, s1s1, s1sM, sMs0, sMs1, sMsM]
    
    param_fourteen = np.empty(14)
    param_fourteen[0] = param_seven_scaled[0]  #s0s0
    param_fourteen[1] = param_seven_scaled[1]/2  #s0s2 (=s0sM/2)
    param_fourteen[2] = param_seven_scaled[1]/2  #s0s3 (=s0sM/2)
    param_fourteen[3] = param_seven_scaled[2]  #s1s1
    param_fourteen[4] = param_seven_scaled[3]/2 #s1s2 (=s1sM)
    param_fourteen[5] = param_seven_scaled[3]/2  #s1s3 
    param_fourteen[6] = param_seven_scaled[4]  #s2s0 (=sMs0)
    param_fourteen[7] = param_seven_scaled[5]  #s2s1 (=sMs1)
    param_fourteen[8] = param_seven_scaled[6]/2  #s2s2 (=sMsM/2)
    param_fourteen[9] = param_seven_scaled[6]/2  #s2s3 (=sMsM/2)
    param_fourteen[10] = param_seven_scaled[4]  #s3s0
    param_fourteen[11] = param_seven_scaled[5]   #s3s1
    param_fourteen[12] = param_seven_scaled[6]/2  #s3s2 (=sMsM/2)
    param_fourteen[13] = param_seven_scaled[6]/2 #s3s3 (=sMsM/2)
    
    return param_fourteen

In [ ]:
def scale_seven_parameters(param_seven): 
    # does the scaling to 1 need to happen before or after symmetry?? I think either gives same result
    
    """
    returns 
        scaled seven parameters arrays from unscaled parameters returned from optimisation...
        ...to have probabilities of transitionng from a state add up to 1
    
    arg
        param_seven = unscaled seven parameters
    """

    param_seven_scaled = []

    sum1 = param_seven[0]+param_seven[1]
    sum2 = param_seven[2]+param_seven[3]
    sum3 = param_seven[4]+param_seven[5]+param_seven[6]

    for i in range(2):
        param_seven_scaled.append(param_seven[i]/sum1)
    for i in range(2,4):
        param_seven_scaled.append(param_seven[i]/sum2) 
    for i in range(4,7):
        param_seven_scaled.append(param_seven[i]/sum3)

 
    return param_seven_scaled

In [ ]:
def weighed_parameters_multiple(param_seven, k, scale=False):
    '''
    returns
        array of transition probababilities where transitions to anterograde/retrograde...
        ...are weighed with individual k values 
    
    args
        param_seven: seven paramters array(non-scaled if result of unconstrained optimsiaiton)
        k: weights - list of 4 values between 0 and 1 
        scale: pass True if param_seven needs to be scaled (default== False)
 
    '''
    ## In terms of concept - need to check s3s2 and s2s3 are correctly labelled , ie are they related or two completely different ks ?
    
    if scale == True:
    
        param_seven_scaled = []

        sum1 = param_seven[0]+param_seven[1]
        sum2 = param_seven[2]+param_seven[3]
        sum3 = param_seven[4]+param_seven[5]+param_seven[6]

        for i in range(2):
            param_seven_scaled.append(param_seven[i]/sum1)
        for i in range(2,4):
            param_seven_scaled.append(param_seven[i]/sum2) 
        for i in range(4,7):
            param_seven_scaled.append(param_seven[i]/sum3)
    
    else:
        param_seven_scaled = np.copy(param_seven)
    
    
    param = np.empty(14)
    param[0] = param_seven_scaled[0]               #s0s0
    param[1] = param_seven_scaled[1]*k[0]          #s0s2 = s0sM*k0
    param[2] = param_seven_scaled[1]*(1-k[0])      #s0s3 = s0sM*(1-k0)
    param[3] = param_seven_scaled[2]               #s1s1 
    param[4] = param_seven_scaled[3]*k[1]          #s1s2 = s1sM*k1
    param[5] = param_seven_scaled[3]*(1-k[1])      #s1s3 = s1sM*(1-k1)
    param[6] = param_seven_scaled[4]               #s2s0 = sMs0
    param[7] = param_seven_scaled[5]               #s2s1 = sMs1
    param[8] = param_seven_scaled[6]*k[2]          #s2s2 = sMsM*k2  
    param[9] = param_seven_scaled[6]*(1-k[2])      #s2s3 = sMsM*(1-k2)
    param[10] = param_seven_scaled[4]              #s3s0 = sMs0
    param[11] = param_seven_scaled[5]              #s3s1 = sMs1
    param[12] = param_seven_scaled[6]*k[3]         #s3s2 = s3s2*k3
    param[13] = param_seven_scaled[6]*(1-k[3])     #s3s3 = sMsM*(1-k3)         

    
    return param

In [ ]:
def weighed_parameters_single(param_seven, k, scale=False):
    '''
    returns 
        array of transition probababilities where transitions to anterograde/retrograde...
        ...are weighed with an individual k value
    
    args
        param_seven: seven paramters array(non-scaled if result of unconstrained optimsiaiton)
        k: single value (float) between 0 and 1 
        scale: pass True if param_seven needs to be scaled (default== False)
    
    '''
    ## In terms of concept - need to check s3s2 and s2s3 are correctly labelled , ie are they related or two completely different ks ?
    ## see PPT slide of states diagrams 
    
    if scale == True:
    
        param_seven_scaled = []

        sum1 = param_seven[0]+param_seven[1]
        sum2 = param_seven[2]+param_seven[3]
        sum3 = param_seven[4]+param_seven[5]+param_seven[6]

        for i in range(2):
            param_seven_scaled.append(param_seven[i]/sum1)
        for i in range(2,4):
            param_seven_scaled.append(param_seven[i]/sum2) 
        for i in range(4,7):
            param_seven_scaled.append(param_seven[i]/sum3)
    
    else:
        param_seven_scaled = np.copy(param_seven)
    
    #first option
    param = np.empty(14)
    param[0] = param_seven_scaled[0]            #s0s0
    param[1] = param_seven_scaled[1]*k          #s0s2 = s0sM*k
    param[2] = param_seven_scaled[1]*(1-k)      #s0s3 = s0sM*(1-k)
    param[3] = param_seven_scaled[2]            #s1s1
    param[4] = param_seven_scaled[3]*k          #s1s2 = s1sM*k
    param[5] = param_seven_scaled[3]*(1-k)      #s1s3 = s1sM*(1-k)
    param[6] = param_seven_scaled[4]            #s2s0 = sMs0
    param[7] = param_seven_scaled[5]            #s2s1 = sMs1
    param[8] = param_seven_scaled[6]*k          #s2s2 = sMsM*k
    param[9] = param_seven_scaled[6]*(1-k)      #s2s3 = sMsM*(1-k)
    param[10] = param_seven_scaled[4]           #s3s0 = sMs0
    param[11] = param_seven_scaled[5]           #s3s1 = sMs1
    param[12] = param_seven_scaled[6]*k         #s3s2 = sMsM*k
    param[13] = param_seven_scaled[6]*(1-k)     #s3s3 = sMsM*(1-k)      
    
    return param

In [ ]:
def weighed_parameters_fourteen_single(param_fourteen, k, scale=False):
    
    '''
    returns 
        array of transition probababilities where transitions to anterograde/retrograde...
        ...are weighed with an individual k value
    
    args
        param_fourteen: fourteen paramters array(non-scaled if result of unconstrained optimsiaiton)
        k: single value (float) between 0 and 1 
        scale: False - param_fourteen should already be scaled
    
    '''
    ## In terms of concept - need to check s3s2 and s2s3 are correctly labelled , ie are they related or two completely different ks ?
    ## see PPT slide of states diagrams 
    
    if scale == True:
    
        param_fourteen_scaled = []

        sum1 = param_fourteen[0]+param_fourteen[1]+param_fourteen[2]
        sum2 = param_fourteen[3]+param_fourteen[4]+param_fourteen[5]
        sum3 = param_fourteen[6]+param_fourteen[7]+param_fourteen[8]+param_fourteen[9]
        sum4 = param_fourteen[10]+param_fourteen[11]+param_fourteen[12]+param_fourteen[13]

        for i in range(3):
            param_fourteen_scaled.append(param_fourteen[i]/sum1)
        for i in range(3,6):
            param_fourteen_scaled.append(param_fourteen[i]/sum2) 
        for i in range(6,10):
            param_fourteen_scaled.append(param_fourteen[i]/sum3)
        for i in range(10,14):
            param_fourteen_scaled.append(param_fourteen[i]/sum4)
    
    else:
        param_fourteen_scaled = np.copy(param_fourteen)
    
    #first option
    param = np.empty(14)
    param[0] = param_fourteen_scaled[0]            #s0s0
    param[1] = param_fourteen_scaled[1]*k          #s0s2 = s0sM*k
    param[2] = param_fourteen_scaled[2]*(1-k)      #s0s3 = s0sM*(1-k)
    param[3] = param_fourteen_scaled[3]            #s1s1
    param[4] = param_fourteen_scaled[4]*k          #s1s2 = s1sM*k
    param[5] = param_fourteen_scaled[5]*(1-k)      #s1s3 = s1sM*(1-k)
    param[6] = param_fourteen_scaled[6]            #s2s0 = sMs0
    param[7] = param_fourteen_scaled[7]            #s2s1 = sMs1
    param[8] = param_fourteen_scaled[8]*k          #s2s2 = sMsM*k
    param[9] = param_fourteen_scaled[9]*(1-k)      #s2s3 = sMsM*(1-k)
    param[10] = param_fourteen_scaled[10]           #s3s0 = sMs0
    param[11] = param_fourteen_scaled[11]           #s3s1 = sMs1
    param[12] = param_fourteen_scaled[12]*k         #s3s2 = sMsM*k
    param[13] = param_fourteen_scaled[13]*(1-k)     #s3s3 = sMsM*(1-k)      
    
    return param

## Import Dstats & CIs from matlab 

In [ ]:
#Define your variables used as target statistics for optimisation (calculated from imaging data)

Dstat1  # P immobile->mobile
Dstat2  # P mobile -> immobile
Dstat3 # meam immobile duration(sec) 
Dstat4 # stdevimmobile duration(sec) 
Dstat5 # 4228-fraction always paused
Dstat6 # Apparent forward -> backward tr prob
Dstat7 # apparent backward -> forward tr prob
Dstat8 #apparent forward -> forward tr prob
Dstat9 # apparent backward -> backward tr prob
Dstat10 # apparent immobile -> backward tr prob
Dstat11 # apparent immobile -> forward tr prob
Dstat12 # apparent backward -> immobile tr prob
Dstat13 # apparent forward -> immobile tr prob
Dstat14 # mean forward run duration
Dstat15 # std forward run duration 
Dstat16 # mean backward run duration
Dstat17 # std backward run duration

In [ ]:
# Define variables of target sttaistics errors (calculated from imaging data)
# (not used in optimisation process but used after optimisaiton to assess optimised values againtst data values +- 95%CI
Derr1 # 95% confidence interval for Dstat1
Derr2 # 95% confidence interval for Dstat2
Derr3 
Derr4 
Derr5
Derr6 
Derr7 
Derr8 
Derr9
Derr10 
Derr11 
Derr12 
Derr13 
Derr14 
Derr15 
Derr16 
Derr17 

## Optimisation functions

In [ ]:
## not used here, this function is for lumped (not split) optimisation strategy. Use objective1 and objective2 functions instead.

def objective(param):
    #s0s0, s0s2, s0s3 , s1s1, s1s2, s1s3, s2s0, s2s1, s2s2, s2s3, s3s0, s3s1, s3s2, s3s3 = param # to unpack here or in sim emsemble?
     
    
    ##run the 'sim_ensemble' function
    X, S  = modsim.sim_ensemble(Parent, ChildrenWeighed, paramfixed, param, scale=True)  # arrays are 2d where row = 1 run and column = timepoint 
    
    ##extract 5 minutes of simulation result
    X1 = np.copy(X[:,100:numtimesteps])
    S1 = np.copy(S[:,100:numtimesteps])
    
    ##Calculate stats from simulation 
        #Stat1&2-Sim
    Smi = modout.states_to_mobility(S1)
    
    tr_i_i, tr_i_m, tr_m_i, tr_m_m = modout.mob_immob_transitions(Smi)
    stat1 = tr_i_m
    stat2 = tr_m_i
    
        #Stat3&4&5-Sim
    immo_lengths = modout.immobile_lengths(Smi,timestep)
    stat3 = np.mean(immo_lengths)
    stat4 = np.std(immo_lengths)
    stat5 = immo_lengths.tolist().count((numtimesteps-100)*timestep)/len(immo_lengths) # fraction of immobile for whole sim
    
        #Stats6-13
    Sfb = modout.forward_backward_states(S1)
    fbtr = modout.forward_backward_transitions(Sfb) # dictionary of forward/backward transitions
    
    stat6 = fbtr['tr_f_b'] 
    stat7 = fbtr['tr_b_f']
    stat8 = fbtr['tr_f_f']
    stat9 = fbtr['tr_b_b']
    stat10 = fbtr['tr_immo_b'] 
    stat11 = fbtr['tr_immo_f'] 
    stat12 = fbtr['tr_b_immo'] 
    stat13 = fbtr['tr_f_immo'] 
    
    
        #Stats 14-17
    forw_length, backw_length = modout.fb_lengths(Sfb, timestep)

    stat14 = np.mean(forw_length)
    stat15 = np.std(forw_length)
    stat16 = np.mean(backw_length)
    stat17 = np.std(backw_length)
    
    
    ##calculate error between sim stats and data stats 
    Error_1 = ((stat1 - Dstat1)/ Dstat1)**2     # P immobile->mobile
    Error_2 = ((stat2 - Dstat2)/ Dstat2)**2 # P mobile -> immobile
    Error_3 = ((stat3 - Dstat3) / Dstat3)**2 # meam immobile duration 
    Error_4 = ((stat4 - Dstat4) / Dstat4)**2 # stdev immobile duration
    Error_5 = ((stat5 - Dstat5) / Dstat5)**2 # fraction of immobile for whole sim
    Error_6 = ((stat6 - Dstat6) / Dstat6)**2 # Apparent forward -> backward tr prob
    Error_7 = ((stat7 - Dstat7)/ Dstat7)**2 # apparent backward -> forward tr prob
    Error_8 = ((stat8 - Dstat8)/ Dstat8)**2 #apparent forward -> forward tr prob
    Error_9 = ((stat9 - Dstat9)/ Dstat9)**2# apparent backward -> backward tr prob
    Error_10 = ((stat10 - Dstat10)/ Dstat10)**2# apparent immobile -> backward tr prob
    Error_11 = ((stat11 - Dstat11)/ Dstat11)**2# apparent immobile -> forward tr prob
    Error_12 = ((stat12 - Dstat12)/ Dstat12)**2# apparent backward -> immobile tr prob
    Error_13 = ((stat13 - Dstat13)/ Dstat13)**2# apparent forward -> immobile tr prob
    Error_14 = ((stat14 - Dstat14)/ Dstat14)**2#mean forward run duration
    Error_15 = ((stat15 - Dstat15)/ Dstat15)**2#std forward run duration 
    Error_16 = ((stat16 - Dstat16)/ Dstat16)**2#mean backward run duration
    Error_17 = ((stat17 - Dstat17)/ Dstat17)**2#std backward run duration
    
    Total_error = Error_1 + Error_2 + Error_3 + Error_4 + Error_5 + Error_6 + Error_7 + Error_8 +Error_9 + Error_10 + Error_11 + Error_12 + Error_13 + Error_14 + Error_15 + Error_16 + Error_17
    
    return Total_error
    


In [ ]:
def objective1(param_seven): 
    """
    returns
        sum of squared errors between data (target) stat and estimated stat from parameters
        
    arg
        param_seven: seven paramters array (scaled - see note below) 
    
    note: seven parameters array should have proababilities of transitioning from one state add up to 1 
            (if this is not the case, need to change symmetrical_parameter function calling to arg scale==True)
    """
    
    ##convert 7 transitions parameters (pause, off-track, moving) into full (14) parameters transition)
    param = symmetrical_parameters(param_seven, scale=False)
    
    #for information:
    #s0s2 = s0s3
    #s2s0 = s3s0
    #s1s2 = s1s3
    #s2s1 = s3s1
    #s2s2 = s3s3
    #s2s3 = s3s3
    #s3s2 = s3s3
    #(s2s2 = s2s3 = s3s2 = s3s3)
    
    #s0s0 = param[0]
    #s0s2 = param[1]
    #s0s3 = param[2]
    #s1s1 = param[3]
    #s1s2 = param[4]
    #s1s3 = param[5]
    #s2s0 = param[6]
    #s2s1 = param[7]
    #s2s2 = param[8]
    #s2s3 = param[9]
    #s3s0 = param[10]
    #s3s1 = param[11]
    #s3s2 = param[12]
    #s3s3 = param[13]
    
    
    
    ##run the 'sim_ensemble' function
    X, S  = modsim.sim_ensemble(Parent, ChildrenWeighed, paramfixed, param, scale=True)  # arrays are 2d where row = 1 run and column = timepoint 
    
    ##extract 5 minutes of simulation result
    X1 = np.copy(X[:,100:numtimesteps])
    S1 = np.copy(S[:,100:numtimesteps])
    
    ##Calculate stats from simulation 
        #Stat1&2-Sim
    Smi = modout.states_to_mobility(S1)
    
    tr_i_i, tr_i_m, tr_m_i, tr_m_m = modout.mob_immob_transitions(Smi)
    stat1 = tr_i_m
    stat2 = tr_m_i
    
        #Stat3&4&5-Sim
    immo_lengths = modout.immobile_lengths(Smi,timestep)
    stat3 = np.mean(immo_lengths)
    stat4 = np.std(immo_lengths)
    stat5 = immo_lengths.tolist().count((numtimesteps-100)*timestep)/len(immo_lengths) # fraction of immobile for whole sim
    
        
    ##calculate error between sim stats and data stats 
    Error_1 = ((stat1 - Dstat1)/ Dstat1)**2     # P immobile->mobile
    Error_2 = ((stat2 - Dstat2)/ Dstat2)**2 # P mobile -> immobile
    Error_3 = ((stat3 - Dstat3) / Dstat3)**2 # meam immobile duration 
    Error_4 = ((stat4 - Dstat4) / Dstat4)**2 # stdev immobile duration
    Error_5 = ((stat5 - Dstat5) / Dstat5)**2 # fraction of immobile for whole sim
    
    
    Total_error = Error_1 + Error_2 + Error_3 + Error_4 + Error_5 
    
    return Total_error 


In [ ]:
def objective1_test(param_seven): 
    """
    returns 
        individual errors (5) and sum of errors - used for plotting errors after optimisation
    
    arg
        param_seven: seven paramters array (scaled - see note below) 
        
    note: seven parameters array should have proababilities of transitioning from one state add up to 1 
            (if this is not the case, need to change symmertical_parameter function calling to arg scale==True)
    """
    
    ##convert 7 transitions parameters (pause, off-track, moving) into full (14) parameters transition)
    param = symmetrical_parameters(param_seven, scale=False)
    
    # for information:
    #s0s2 = s0s3
    #s2s0 = s3s0
    #s1s2 = s1s3
    #s2s1 = s3s1
    #s2s2 = s3s3
    #s2s3 = s3s3
    #s3s2 = s3s3
    #(s2s2 = s2s3 = s3s2 = s3s3)
    
    
    #s0s0 = param[0]
    #s0s2 = param[1]
    #s0s3 = param[2]
    #s1s1 = param[3]
    #s1s2 = param[4]
    #s1s3 = param[5]
    #s2s0 = param[6]
    #s2s1 = param[7]
    #s2s2 = param[8]
    #s2s3 = param[9]
    #s3s0 = param[10]
    #s3s1 = param[11]
    #s3s2 = param[12]
    #s3s3 = param[13]
    
    
    ##run the 'sim_ensemble' function
    X, S  = modsim.sim_ensemble(Parent, ChildrenWeighed, paramfixed, param, scale=True)  # arrays are 2d where row = 1 run and column = timepoint 
    
    ##extract 5 minutes of simulation result
    X1 = np.copy(X[:,100:numtimesteps])
    S1 = np.copy(S[:,100:numtimesteps])
    
    ##Calculate stats from simulation 
        #Stat1&2-Sim
    Smi = modout.states_to_mobility(S1)
    
    tr_i_i, tr_i_m, tr_m_i, tr_m_m = modout.mob_immob_transitions(Smi)
    stat1 = tr_i_m
    stat2 = tr_m_i
    
        #Stat3&4&5-Sim
    immo_lengths = modout.immobile_lengths(Smi,timestep)
    stat3 = np.mean(immo_lengths)
    stat4 = np.std(immo_lengths)
    stat5 = immo_lengths.tolist().count((numtimesteps-100)*timestep)/len(immo_lengths) # fraction of immobile for whole sim
    
    
    ##calculate error between sim stats and data stats 
    Error_1 = ((stat1 - Dstat1)/ Dstat1)**2     # P immobile->mobile
    Error_2 = ((stat2 - Dstat2)/ Dstat2)**2 # P mobile -> immobile
    Error_3 = ((stat3 - Dstat3) / Dstat3)**2 # meam immobile duration 
    Error_4 = ((stat4 - Dstat4) / Dstat4)**2 # stdev immobile duration
    Error_5 = ((stat5 - Dstat5) / Dstat5)**2 # fraction of immobile for whole sim
    
    
    Total_error = Error_1 + Error_2 + Error_3 + Error_4 + Error_5 
    
    return Error_1, Error_2, Error_3, Error_4, Error_5, Total_error 


In [ ]:
def objective2(k): 
    
    '''
    gloabl variables requirements: 
        'result_param_seven_opti_scaled' -> seven parameter array from first step of optimisation & scaled
    
    returns 
        individual errors (5) and sum of errors - used for plotting errors after optimisation
    
    arg
        k: weights - list of 4 values between 0 and 1 
        
    note: seven parameters array should have proababilities of transitioning from one state add up to 1 
            (if this is not the case, need to change symmertical_parameter function calling to arg scale==True)
    
    '''
    
    #weighed aneterograde/retrograde transition probabilities  with new definitions - refer to powerpoit state diagram
    param = weighed_parameters_multiple(result_param_seven_opti_scaled, k, scale=False) # transition
    

    
    ##run the 'sim_ensemble' function
    X, S  = modsim.sim_ensemble(Parent, ChildrenWeighed, paramfixed, param, scale=True)  # arrays are 2d where row = 1 run and column = timepoint 
    
    ##extract 5 minutes of simulation result
    X1 = np.copy(X[:,100:numtimesteps])
    S1 = np.copy(S[:,100:numtimesteps])
    

        #Stats6-13
    Sfb = modout.forward_backward_states(S1)
    fbtr = modout.forward_backward_transitions(Sfb) # dictionary of forward/backward transitions
    
    stat6 = fbtr['tr_f_b'] 
    stat7 = fbtr['tr_b_f']
    stat8 = fbtr['tr_f_f']
    stat9 = fbtr['tr_b_b'] 
    stat10 = fbtr['tr_immo_b'] 
    stat11 = fbtr['tr_immo_f'] 
    stat12 = fbtr['tr_b_immo'] 
    stat13 = fbtr['tr_f_immo'] 
    
        #Stats 14-17
    forw_length, backw_length = modout.fb_lengths(Sfb,timestep)

    stat14 = np.mean(forw_length)
    stat15 = np.std(forw_length)
    stat16 = np.mean(backw_length)
    stat17 = np.std(backw_length)
    
    
    ##calculate error between sim stats and data stats 
    
    Error_6 = ((stat6 - Dstat6) / Dstat6)**2 # Apparent forward -> backward tr prob
    Error_7 = ((stat7 - Dstat7)/ Dstat7)**2 # apparent backward -> forward tr prob
    Error_8 = ((stat8 - Dstat8)/ Dstat8)**2 #apparent forward -> forward tr prob
    Error_9 = ((stat9 - Dstat9)/ Dstat9)**2 # apparent backward -> backward tr prob
    Error_10 = ((stat10 - Dstat10)/ Dstat10)**2# apparent immobile -> backward tr prob
    Error_11 = ((stat11 - Dstat11)/ Dstat11)**2# apparent immobile -> forward tr prob
    Error_12 = ((stat12 - Dstat12)/ Dstat12)**2# apparent backward -> immobile tr prob
    Error_13 = ((stat13 - Dstat13)/ Dstat13)**2# apparent forward -> immobile tr prob
    Error_14 = ((stat14 - Dstat14)/ Dstat14)**2#mean forward run duration
    Error_15 = ((stat15 - Dstat15)/ Dstat15)**2#std forward run duration 
    Error_16 = ((stat16 - Dstat16)/ Dstat16)**2#mean backward run duration
    Error_17 = ((stat17 - Dstat17)/ Dstat17)**2#std backward run duration
    
    Total_error = Error_6 + Error_7 + Error_8 +Error_9 + Error_10 + Error_11 + Error_12 + Error_13 + Error_14 + Error_15 + Error_16 + Error_17
    
    return Total_error 


In [ ]:
def objective2_test(param): 
    
    '''
    returns 
        individual errors (12) and sum of errors - used for plotting errors after optimisation
    
    arg
        param: final results parameters (need to be scaled)
    '''
    
    ##run the 'sim_ensemble' function
    X, S  = modsim.sim_ensemble(Parent, ChildrenWeighed, paramfixed, param, scale=True)  # arrays are 2d where row = 1 run and column = timepoint 
    
    ##extract 5 minutes of simulation result
    X1 = np.copy(X[:,100:numtimesteps])
    S1 = np.copy(S[:,100:numtimesteps])
    

        #Stats6-13
    Sfb = modout.forward_backward_states(S1)
    fbtr = modout.forward_backward_transitions(Sfb) # dictionary of forward/backward transitions
    
    stat6 = fbtr['tr_f_b'] 
    stat7 = fbtr['tr_b_f']
    stat8 = fbtr['tr_f_f']
    stat9 = fbtr['tr_b_b']
    stat10 = fbtr['tr_immo_b'] 
    stat11 = fbtr['tr_immo_f'] 
    stat12 = fbtr['tr_b_immo'] 
    stat13 = fbtr['tr_f_immo'] 
    
    
        #Stats 14-17
    forw_length, backw_length = modout.fb_lengths(Sfb,timestep)

    stat14 = np.mean(forw_length)
    stat15 = np.std(forw_length)
    stat16 = np.mean(backw_length)
    stat17 = np.std(backw_length)

    
    ##calculate error between sim stats and data stats 
    
    Error_6 = ((stat6 - Dstat6) / Dstat6)**2 # Apparent forward -> backward tr prob
    Error_7 = ((stat7 - Dstat7)/ Dstat7)**2 # apparent backward -> forward tr prob
    Error_8 = ((stat8 - Dstat8)/ Dstat8)**2 #apparent forward -> forward tr prob
    Error_9 = ((stat9 - Dstat9)/ Dstat9)**2 # apparent backward -> backward tr prob
    Error_10 = ((stat10 - Dstat10)/ Dstat10)**2# apparent immobile -> backward tr prob
    Error_11 = ((stat11 - Dstat11)/ Dstat11)**2# apparent immobile -> forward tr prob
    Error_12 = ((stat12 - Dstat12)/ Dstat12)**2# apparent backward -> immobile tr prob
    Error_13 = ((stat13 - Dstat13)/ Dstat13)**2# apparent forward -> immobile tr prob
    Error_14 = ((stat14 - Dstat14)/ Dstat14)**2#mean forward run duration
    Error_15 = ((stat15 - Dstat15)/ Dstat15)**2#std forward run duration 
    Error_16 = ((stat16 - Dstat16)/ Dstat16)**2#mean backward run duration
    Error_17 = ((stat17 - Dstat17)/ Dstat17)**2#std backward run duration
    
    Total_error = Error_6 + Error_7 + Error_8 +Error_9 + Error_10 + Error_11 + Error_12 + Error_13 + Error_14 + Error_15 + Error_16 + Error_17
    
    return Error_6, Error_7, Error_8, Error_9, Error_10, Error_11, Error_12, Error_13, Error_14, Error_15, Error_16, Error_17, Total_error 


In [ ]:
def objective2_single(k): #param_six
    
    '''
    global variables requirements: 
        'result_param_seven_opti_scaled' -> seven parameter array from first step of optimisation & scaled
    
    returns 
        individual errors (5) and sum of errors - used for plotting errors after optimisation
    
    arg
        k: weight - single k value
        
    note: seven parameters array should have proababilities of transitioning from one state add up to 1 
            (if this is not the case, need to change symmertical_parameter function calling to arg scale==True)

    '''
    
    #weighed aneterograde/retrograde transition probabilities  with new deifnitions - refer to powerpoit state diagram
    param = weighed_parameters_single(result_param_seven_opti_scaled, k, scale=False)

    
    ##run the 'sim_ensemble' function
    X, S  = modsim.sim_ensemble(Parent, ChildrenWeighed, paramfixed, param, scale=True)  # arrays are 2d where row = 1 run and column = timepoint 
    
    ##extract 5 minutes of simulation result
    X1 = np.copy(X[:,100:numtimesteps])
    S1 = np.copy(S[:,100:numtimesteps])
    

        #Stats6-13
    Sfb = modout.forward_backward_states(S1)
    fbtr = modout.forward_backward_transitions(Sfb) # dictionary of forward/backward transitions
    
    stat6 = fbtr['tr_f_b'] 
    stat7 = fbtr['tr_b_f']
    stat8 = fbtr['tr_f_f']
    stat9 = fbtr['tr_b_b']
    stat10 = fbtr['tr_immo_b'] 
    stat11 = fbtr['tr_immo_f'] 
    stat12 = fbtr['tr_b_immo'] 
    stat13 = fbtr['tr_f_immo'] 
    
    
        #Stats 14-17
    forw_length, backw_length = modout.fb_lengths(Sfb,timestep)

    stat14 = np.mean(forw_length)
    stat15 = np.std(forw_length)
    stat16 = np.mean(backw_length)
    stat17 = np.std(backw_length)
    
    
    
    ##calculate error between sim stats and data stats 
    
    Error_6 = ((stat6 - Dstat6) / Dstat6)**2 # Apparent forward -> backward tr prob
    Error_7 = ((stat7 - Dstat7)/ Dstat7)**2 # apparent backward -> forward tr prob
    Error_8 = ((stat8 - Dstat8)/ Dstat8)**2 #apparent forward -> forward tr prob
    Error_9 = ((stat9 - Dstat9)/ Dstat9)**2 # apparent backward -> backward tr prob
    Error_10 = ((stat10 - Dstat10)/ Dstat10)**2# apparent immobile -> backward tr prob
    Error_11 = ((stat11 - Dstat11)/ Dstat11)**2# apparent immobile -> forward tr prob
    Error_12 = ((stat12 - Dstat12)/ Dstat12)**2# apparent backward -> immobile tr prob
    Error_13 = ((stat13 - Dstat13)/ Dstat13)**2# apparent forward -> immobile tr prob
    Error_14 = ((stat14 - Dstat14)/ Dstat14)**2#mean forward run duration
    Error_15 = ((stat15 - Dstat15)/ Dstat15)**2#std forward run duration 
    Error_16 = ((stat16 - Dstat16)/ Dstat16)**2#mean backward run duration
    Error_17 = ((stat17 - Dstat17)/ Dstat17)**2#std backward run duration
    
    Total_error = Error_6 + Error_7 + Error_8 +Error_9 + Error_10 + Error_11 + Error_12 + Error_13 + Error_14 + Error_15 + Error_16 + Error_17
    
    return Total_error 


In [ ]:
def stats_optioutput(param):
        
    ##run the 'sim_ensemble' function
    X, S  = modsim.sim_ensemble(Parent, ChildrenWeighed, paramfixed, param, scale=True)  # arrays are 2d where row = 1 run and column = timepoint 
    
    ##extract 5 minutes of simulation result
    X1 = np.copy(X[:,100:numtimesteps])
    S1 = np.copy(S[:,100:numtimesteps])
    
    ##Calculate stats from simulation 
        #Stat1&2-Sim
    
    Smi = modout.states_to_mobility(S1)
    
    tr_i_i, tr_i_m, tr_m_i, tr_m_m = modout.mob_immob_transitions(Smi)
    stat1 = tr_i_m
    stat2 = tr_m_i
    
        #Stat3&4&5-Sim
    immo_lengths = modout.immobile_lengths(Smi,timestep)
    stat3 = np.mean(immo_lengths)
    stat4 = np.std(immo_lengths)
    stat5 = immo_lengths.tolist().count((numtimesteps-100)*timestep)/len(immo_lengths) # fraction of immobile for whole sim

        #Stats6-13
    Sfb = modout.forward_backward_states(S1)
    fbtr = modout.forward_backward_transitions(Sfb) # dictionary of forward/backward transitions
    
    stat6 = fbtr['tr_f_b'] 
    stat7 = fbtr['tr_b_f']
    stat8 = fbtr['tr_f_f']
    stat9 = fbtr['tr_b_b']
    stat10 = fbtr['tr_immo_b'] 
    stat11 = fbtr['tr_immo_f'] 
    stat12 = fbtr['tr_b_immo'] 
    stat13 = fbtr['tr_f_immo'] 
    
    
        #Stats 14-17
    forw_length, backw_length = modout.fb_lengths(Sfb,timestep)

    stat14 = np.mean(forw_length)
    stat15 = np.std(forw_length)
    stat16 = np.mean(backw_length)
    stat17 = np.std(backw_length)
 
    
    simoutputstats = {'stat1':stat1, 'stat2':stat2, 'stat3':stat3, 'stat4':stat4, 'stat5':stat5, 'stat6':stat6, 'stat7':stat7,
                      'stat8':stat8,'stat9':stat9,'stat10':stat10, 'stat11':stat11, 'stat12':stat12, 'stat13':stat13, 
                      'stat14':stat14, 'stat15':stat15, 'stat16':stat16, 'stat17':stat17}
    
    
    return simoutputstats
    

In [ ]:
def stats_optioutput_plus(param):
    
    ##run the 'sim_ensemble' function
    X, S  = modsim.sim_ensemble(Parent, ChildrenWeighed, paramfixed, param, scale=True)  # arrays are 2d where row = 1 run and column = timepoint 
    
    ##extract 5 minutes of simulation result
    X1 = np.copy(X[:,100:numtimesteps])
    S1 = np.copy(S[:,100:numtimesteps])
    
    ##Calculate stats from simulation 
        #Stat1&2-Sim
    Smi = modout.states_to_mobility(S1)
    
    tr_i_i, tr_i_m, tr_m_i, tr_m_m = modout.mob_immob_transitions(Smi)
    stat1 = tr_i_m
    stat2 = tr_m_i
    
        #Stat3&4&5-Sim
    immo_lengths = modout.immobile_lengths(Smi,timestep)
    stat3 = np.mean(immo_lengths)
    stat4 = np.std(immo_lengths)
    stat5 = immo_lengths.tolist().count((numtimesteps-100)*timestep)/len(immo_lengths) # fraction of immobile for whole sim
    
        #Stats6-13
    Sfb = modout.forward_backward_states(S1)
    fbtr = modout.forward_backward_transitions(Sfb) # dictionary of forward/backward transitions
    
    stat6 = fbtr['tr_f_b'] 
    stat7 = fbtr['tr_b_f']
    stat8 = fbtr['tr_f_f']
    stat9 = fbtr['tr_b_b']
    stat10 = fbtr['tr_immo_b'] 
    stat11 = fbtr['tr_immo_f'] 
    stat12 = fbtr['tr_b_immo'] 
    stat13 = fbtr['tr_f_immo'] 
    
    
        #Stats 14-17
    forw_length, backw_length = modout.fb_lengths(Sfb,timestep)

    stat14 = np.mean(forw_length)
    stat15 = np.std(forw_length)
    stat16 = np.mean(backw_length)
    stat17 = np.std(backw_length)
 
    
    simoutputstats = {'stat1':stat1, 'stat2':stat2, 'stat3':stat3, 'stat4':stat4, 'stat5':stat5, 'stat6':stat6, 'stat7':stat7,
                      'stat8':stat8,'stat9':stat9,'stat10':stat10, 'stat11':stat11, 'stat12':stat12, 'stat13':stat13, 
                      'stat14':stat14, 'stat15':stat15, 'stat16':stat16, 'stat17':stat17}
    
    
    return simoutputstats, backw_length, forw_length
    

In [ ]:
def fb_lengths_track(fb_array,tstep):
    fw_lengths = modout.count_consecutive(fb_array, 5)
    forward_lengths_sec = list((np.copy(fw_lengths))*tstep)      #transform values into seconds 
    
    bw_lengths = modout.count_consecutive(fb_array, 4)
    backward_lengths_sec = list((np.copy(bw_lengths))*tstep)      #transform values into seconds 
    
    fw_num = len(forward_lengths_sec) #number of forward runs
    bw_num = len(backward_lengths_sec)
    
    if fw_num!=0:
        fw_length_mean = np.nanmean(forward_lengths_sec) # mean forward length
    else:
        fw_length_mean = np.nan
    
    if bw_num!=0:
        bw_length_mean = np.nanmean(backward_lengths_sec)
    else: 
        bw_length_mean = np.nan
    
    return fw_num, bw_num, fw_length_mean, bw_length_mean

In [ ]:
def stats_optioutput_plus(param):
    
    ##run the 'sim_ensemble' function
    X, S  = modsim.sim_ensemble(Parent, ChildrenWeighed, paramfixed, param, scale=True)  # arrays are 2d where row = 1 run and column = timepoint 
    
    ##extract 5 minutes of simulation result
    X1 = np.copy(X[:,100:numtimesteps])
    S1 = np.copy(S[:,100:numtimesteps])
    
    ##Calculate stats from simulation 
        #Stat1&2-Sim
    Smi = modout.states_to_mobility(S1)
    
    tr_i_i, tr_i_m, tr_m_i, tr_m_m = modout.mob_immob_transitions(Smi)
    stat1 = tr_i_m
    stat2 = tr_m_i
    
        #Stat3&4&5-Sim
    immo_lengths = modout.immobile_lengths(Smi,timestep)
    stat3 = np.mean(immo_lengths)
    stat4 = np.std(immo_lengths)
    stat5 = immo_lengths.tolist().count((numtimesteps-100)*timestep)/len(immo_lengths) # fraction of immobile for whole sim
    
        #Stats6-13
    Sfb = modout.forward_backward_states(S1)
    fbtr = modout.forward_backward_transitions(Sfb) # dictionary of forward/backward transitions
    
    stat6 = fbtr['tr_f_b'] 
    stat7 = fbtr['tr_b_f']
    stat8 = fbtr['tr_f_f']
    stat9 = fbtr['tr_b_b']
    stat10 = fbtr['tr_immo_b'] 
    stat11 = fbtr['tr_immo_f'] 
    stat12 = fbtr['tr_b_immo'] 
    stat13 = fbtr['tr_f_immo'] 
    
        #Stats 14-17
    forw_length, backw_length = modout.fb_lengths(Sfb,timestep)

    stat14 = np.mean(forw_length)
    stat15 = np.std(forw_length)
    stat16 = np.mean(backw_length)
    stat17 = np.std(backw_length)
    
    #forward-backward time per track
    Fpertrack = np.empty(len(Sfb))
    Bpertrack = np.empty(len(Sfb))
    Flengthtrack = np.empty(len(Sfb))
    Blengthtrack = np.empty(len(Sfb))
    
    for i in range(len(Sfb)):
        Fpertrack[i] = sum(Sfb[i]==5)*timestep
        Bpertrack[i] = sum(Sfb[i]==4)*timestep
    
    #number of forward/backward runs
    
    fwnum = np.empty(len(Sfb))
    bwnum = np.empty(len(Sfb))
    flmean = np.empty(len(Sfb))
    blmean = np.empty(len(Sfb))
    
    for i in range(len(Sfb)):
        res = fb_lengths_track(Sfb[i],timestep)
        fwnum[i] = res[0]
        bwnum[i] = res[1]
        flmean[i] = res[2]
        blmean[i] = res[3]

    
    
    simoutputstats = {'stat1':stat1, 'stat2':stat2, 'stat3':stat3, 'stat4':stat4, 'stat5':stat5, 'stat6':stat6, 'stat7':stat7,
                      'stat8':stat8,'stat9':stat9,'stat10':stat10, 'stat11':stat11, 'stat12':stat12, 'stat13':stat13, 
                      'stat14':stat14, 'stat15':stat15, 'stat16':stat16, 'stat17':stat17}
    
    
    return simoutputstats, backw_length, forw_length, Bpertrack, Fpertrack, bwnum, fwnum, blmean, flmean
    

## Optimisation CMA-ES

In [ ]:
# 1st optimisation step: only considereng transitions between moving, pausing and off-track
# Parameters for 1st step of optimisation = param_seven 

#initial estimated transition parameters:
if genotype == 'wt':
    param_seven_0 = [0.9983, 0.0017, 0.7096, 0.2904, 0.2912, 0.4105, 0.2983]# WT initial parameters - taken from data with arbitrary off-track threhsollde of 48tps 
if genotype == 'tau':
    param_seven_0 = [0.9988, 0.0012, 0.7641, 0.2359, 0.3791, 0.3898, 0.2311] # TG initial parameters - taken from data with arbitrary off-track threhsollde of 48tps 

sigma0 = 1  # initial standard deviation to sample new solutions

#Optimisation:
optiparam1, es1 = cma.fmin2(objective1, param_seven_0, sigma0, {'bounds': [0,None]})
# optimiser is unconstrained - so don't want bounds for values either, then manually scale probabilities to be equal to  1 after unconstrained optimiation results


In [ ]:
es1.plot()

In [ ]:
## Second optimisation step: direction of transport

## enter results of 1st optimisation here (not scaled if unconstrained optimisation)
result_param_seven_opti = [8.22460486e-01, 2.91345744e-06, 1.79508367e-03, 5.03262508e+00, 2.36216914e-04, 1.47111305e+00, 9.48665919e-01] # wt v4.1

# scaled result seven parameters
result_param_seven_opti_scaled = scale_seven_parameters(result_param_seven_opti) 
 
# inital parameters 
param_four_0 = [0.5, 0.5, 0.5, 0.5] # initial weight parameters
sigma0 = 1 # initial standard deviation to sample new solutions

#run optimisation
optiparam2, es2 = cma.fmin2(objective2, param_four_0, sigma0, {'bounds': [0,1]})


In [ ]:
es2.plot()

## Other method: Computing objective 2 function with single k value

In [ ]:
karray = np.arange(0.01,1.0,0.01) # array form 0 to 1 (or 1.05) in 0.01 increments 

obj1best = [8.22460486e-01, 2.91345744e-06, 1.79508367e-03, 5.03262508e+00, 2.36216914e-04, 1.47111305e+00, 9.48665919e-01] #v4.1

# scaled result seven parameters (to be between 0 and 1)
result_param_seven_opti_scaled = scale_seven_parameters(obj1best)



In [ ]:
## single k value computing objective 2

karray = np.arange(0.00,1.01,0.01) # array form 0 to 1 (or 1.05) in 0.01 increments 

objective2_means_range = []
objective2_stdevs_range = []

for k in karray:
    objective2_i = []
    for i in range(100): # runs 100 sim_ensemble per k values and takes mean of those 
        objective2_i.append(objective2_single(k))
        mean_i = np.nanmean(objective2_i)
        stdev_i = np.nanstd(objective2_i)
    objective2_means_range.append(mean_i)
    objective2_stdevs_range.append(stdev_i)



In [ ]:
karray = np.arange(0.00,1.01,0.01) # array form 0 to 1 (or 1.05) in 0.01 increments


fig, ax = plt.subplots()
fig.set_size_inches(9*cm, 8*cm)

ax.scatter(karray, objective2_means_range, color='tab:blue', s=10)


ax.set_xlabel('k')
ax.set_ylabel('Mean total relative error squared')
plt.xticks(np.arange(min(karray), max(karray)+0.1, 0.2))


modplot.set_axes_properties(ax)

In [ ]:
# this works for lists 
singlek_opti = karray[objective2_means_range.index(np.nanmin(objective2_means_range[0:50]))] # k avlue corresponding to minimum error 
print(singlek_opti)


In [ ]:
# this works for arrays 
singlek_opti = karray[np.where(objective2_means_range == np.nanmin(objective2_means_range))] # k avlue corresponding to minimum error 
singlek_opti = karray[np.where(objective2_means_range == np.nanmin(objective2_means_range[0:50]))] # k avlue corresponding to minimum error 
print(singlek_opti)

## Calculating transition probabilities from optimsiation results

In [ ]:
#input result (xbest) from 1st optimisation (seven parameters - mobility)#

obj1best = [8.22460486e-01, 2.91345744e-06, 1.79508367e-03, 5.03262508e+00, 2.36216914e-04, 1.47111305e+00, 9.48665919e-01] #v4.1

# scaled result seven parameters (to be between 0 and 1)
result_param_seven_opti_scaled = scale_seven_parameters(obj1best)

#inpput result (xbest) from 2nd optimisation (four parameters - antero/retro)#
obj2best = [0.97216936, 0.95564403, 0.99479759, 0.82272686] #v4.1

##parameters after Obj_2 multiple optimisation
finalparameters = weighed_parameters_multiple(result_param_seven_opti_scaled, obj2best, scale=False) 
print(finalparameters)

In [ ]:
#parameters from obj1 optimised with equal ks
k_equal = [0.5, 0.5, 0.5, 0.5]
obj1_fourteen_param = weighed_parameters_multiple(result_param_seven_opti_scaled, k_equal, scale=True)
print(obj1_fourteen_param)

In [ ]:
#final paramters after single k value optimisation 
obj2_single_param = weighed_parameters_single(result_param_seven_opti_scaled, singlek_opti, scale=False)
print(obj2_single_param)
print(obj2_single_param[0]+obj2_single_param[1]+obj2_single_param[2])

## Plotting errors results 

In [ ]:
##Simulation parameters: - can be different from ones used in optimation itself
numtimesteps = 223+100 #  # number of timsteps per particle run
runs = 1000  #number of mitochondria 

#Re-compute initial estimated parameters if needed
if genotype == 'wt':
    param_seven_0 = [0.9983, 0.0017, 0.7096, 0.2904, 0.2912, 0.4105, 0.2983]# WT initial parameters - taken from data with arbitrary off-track threhsollde of 48tps 
if genotype == 'tau':
    param_seven_0 = [0.9988, 0.0012, 0.7641, 0.2359, 0.3791, 0.3898, 0.2311] # TG initial parameters - taken from data with arbitrary off-track threhsollde of 48tps 



In [ ]:
## Objective 1 - calculate errors (and variability) of non-optimsed vs optimised parameters
e1_list = []
e2_list = []
e3_list = []
e4_list = []
e5_list = []
tot_list = []

for i in range(100):
    e1, e2, e3, e4, e5, toterrs = objective1_test(param_seven_0)
    e1_list.append(e1)
    e2_list.append(e2)
    e3_list.append(e3)
    e4_list.append(e4)
    e5_list.append(e5)
    tot_list.append(toterrs)


mean_e1 = mean(e1_list)
mean_e2 = mean(e2_list)
mean_e3 = mean(e3_list)
mean_e4 = mean(e4_list)
mean_e5 = mean(e5_list)
mean_tot = mean(tot_list)

stdev_e1 = np.std(e1_list)
stdev_e2 = np.std(e2_list)
stdev_e3 = np.std(e3_list)
stdev_e4 = np.std(e4_list)
stdev_e5 = np.std(e5_list)
stdev_tot = np.std(tot_list)


#Optimised parameters 
e1_list_op = []
e2_list_op = []
e3_list_op = []
e4_list_op = []
e5_list_op = []
tot_list_op = []

for i in range(100):
    e1_op, e2_op, e3_op, e4_op, e5_op, toterrs_op = objective1_test(result_param_seven_opti_scaled)
    e1_list_op.append(e1_op)
    e2_list_op.append(e2_op)
    e3_list_op.append(e3_op)
    e4_list_op.append(e4_op)
    e5_list_op.append(e5_op)
    tot_list_op.append(toterrs_op)

mean_e1_op = mean(e1_list_op)
mean_e2_op = mean(e2_list_op)
mean_e3_op = mean(e3_list_op)
mean_e4_op = mean(e4_list_op)
mean_e5_op = mean(e5_list_op)
mean_tot_op = mean(tot_list_op)

stdev_e1_op = np.std(e1_list_op)
stdev_e2_op = np.std(e2_list_op)
stdev_e3_op = np.std(e3_list_op)
stdev_e4_op = np.std(e4_list_op)
stdev_e5_op = np.std(e5_list_op)
stdev_tot_op = np.std(tot_list_op)

In [ ]:
#Put into lists
mean_e_obj1 =  [mean_e1, mean_e2, mean_e3, mean_e4, mean_e5]
mean_eop_obj1 = [mean_e1_op, mean_e2_op, mean_e3_op, mean_e4_op, mean_e5_op]
stdev_e_obj1 = [stdev_e1, stdev_e2, stdev_e3, stdev_e4, stdev_e5]
stdev_eop_obj1 = [stdev_e1_op, stdev_e2_op, stdev_e3_op, stdev_e4_op, stdev_e5_op]

In [ ]:
cm = 1/2.54 

In [ ]:
## Objective 1 - plot calculate errors
mean_e_obj1 =  [mean_e1, mean_e2, mean_e3, mean_e4, mean_e5]
mean_eop_obj1 = [mean_e1_op, mean_e2_op, mean_e3_op, mean_e4_op, mean_e5_op]


# Calculate the x-axis positions for each set of bars
x = np.arange(len(mean_e_obj1))

fig, ax = plt.subplots()
fig.set_size_inches(9*cm, 8*cm)

# Plot the bars side by side
ax.bar(x - 0.2, mean_e_obj1, yerr=stdev_e_obj1, width=0.4, color='tab:blue', align='center', label='estimated')
ax.bar(x + 0.2, mean_eop_obj1, yerr=stdev_eop_obj1, width=0.4, color='orange', align='center', label='optimised')

ax.set_yscale('log')
ax.set_ylabel('Mean relative error squared')

custom_labels = ['Error 1', 'Error 2', 'Error 3', 'Error 4', 'Error 5']
ax.set_xticks(x)
ax.set_xticklabels(custom_labels,rotation=45)
ax.tick_params(axis='x', which='both', bottom=False, top=False)

ax.legend(loc='lower center', bbox_to_anchor=(0.5, -0.45), ncol=2, prop={'size': 9})


modplot.set_axes_properties(ax)
plt.tick_params(bottom = False) 


In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(5*cm, 8*cm)


ax.bar(['Estimated', ' Optimised'], [mean_tot, mean_tot_op], yerr = [stdev_tot, stdev_tot_op], color=['tab:blue','orange'])
#ax.set_ylabel('Sum of mean relative errors squared')
ax.set_ylabel('Mean total relative error squared')
ax.tick_params(axis='x', which=u'both',length=0) # hide x axis ticks 
#plt.yscale('log')
modplot.set_axes_properties(ax)
plt.tick_params(bottom = False) 


In [ ]:
# Objective 2 - plot calculate errors 
## Objective 1 - calculate errors (and variability) of non-optimsed vs optimised parameters

#Non optimised paramters 
e6_list = []
e7_list = []
e8_list = []
e9_list = []
e10_list = []
e11_list = []
e12_list = []
e13_list = []
e14_list = []
e15_list = []
e16_list = []
e17_list = []
tot2_list = []

for i in range(100):
    e6, e7, e8, e9, e10, e11, e12, e13, e14, e15, e16, e17, toterrs2 = objective2_test(obj1_fourteen_param)
    e6_list.append(e6)
    e7_list.append(e7)
    e8_list.append(e8)
    e9_list.append(e9)
    e10_list.append(e10)
    e11_list.append(e11)
    e12_list.append(e12)
    e13_list.append(e13)
    e14_list.append(e14)
    e15_list.append(e15)
    e16_list.append(e16)
    e17_list.append(e17)
    tot2_list.append(toterrs2)


mean_e6 = mean(e6_list)
mean_e7 = mean(e7_list)
mean_e8 = mean(e8_list)
mean_e9 = mean(e9_list)
mean_e10 = mean(e10_list)
mean_e11 = mean(e11_list)
mean_e12 = mean(e12_list)
mean_e13 = mean(e13_list)
mean_e14 = mean(e14_list)
mean_e15 = mean(e15_list)
mean_e16 = mean(e16_list)
mean_e17 = mean(e17_list)
mean_tot2 = mean(tot2_list)

stdev_e6 = np.std(e6_list)
stdev_e7 = np.std(e7_list)
stdev_e8 = np.std(e8_list)
stdev_e9 = np.std(e9_list)
stdev_e10 = np.std(e10_list)
stdev_e11 = np.std(e11_list)
stdev_e12 = np.std(e12_list)
stdev_e13 = np.std(e13_list)
stdev_e14 = np.std(e14_list)
stdev_e15 = np.std(e15_list)
stdev_e16 = np.std(e16_list)
stdev_e17 = np.std(e17_list)
stdev_tot2 = np.std(tot2_list)


#Optimised parameters 
e6_list_op = []
e7_list_op = []
e8_list_op = []
e9_list_op = []
e10_list_op = []
e11_list_op = []
e12_list_op = []
e13_list_op = []
e14_list_op = []
e15_list_op = []
e16_list_op = []
e17_list_op = []
tot2_list_op = []

for i in range(100):
    e6_op, e7_op, e8_op, e9_op, e10_op, e11_op, e12_op, e13_op, e14_op, e15_op, e16_op, e17_op, toterrs2_op = objective2_test(finalparameters)#
    e6_list_op.append(e6_op)
    e7_list_op.append(e7_op)
    e8_list_op.append(e8_op)
    e9_list_op.append(e9_op)
    e10_list_op.append(e10_op)
    e11_list_op.append(e11_op)
    e12_list_op.append(e12_op)
    e13_list_op.append(e13_op)
    e14_list_op.append(e14_op)
    e15_list_op.append(e15_op)
    e16_list_op.append(e16_op)
    e17_list_op.append(e17_op)
    tot2_list_op.append(toterrs2_op)


mean_e6_op = mean(e6_list_op)
mean_e7_op = mean(e7_list_op)
mean_e8_op = mean(e8_list_op)
mean_e9_op = mean(e9_list_op)
mean_e10_op = mean(e10_list_op)
mean_e11_op = mean(e11_list_op)
mean_e12_op = mean(e12_list_op)
mean_e13_op = mean(e13_list_op)
mean_e14_op = mean(e14_list_op)
mean_e15_op = mean(e15_list_op)
mean_e16_op = mean(e16_list_op)
mean_e17_op = mean(e17_list_op)
mean_tot2_op = mean(tot2_list_op)

stdev_e6_op = np.std(e6_list_op)
stdev_e7_op = np.std(e7_list_op)
stdev_e8_op = np.std(e8_list_op)
stdev_e9_op = np.std(e9_list_op)
stdev_e10_op = np.std(e10_list_op)
stdev_e11_op = np.std(e11_list_op)
stdev_e12_op = np.std(e12_list_op)
stdev_e13_op = np.std(e13_list_op)
stdev_e14_op = np.std(e14_list_op)
stdev_e15_op = np.std(e15_list_op)
stdev_e16_op = np.std(e16_list_op)
stdev_e17_op = np.std(e17_list_op)
stdev_tot2_op = np.std(tot2_list_op)

In [ ]:
#Put into lists
mean_e_obj2 =  [mean_e6, mean_e7, mean_e8, mean_e9, mean_e10, mean_e11, mean_e12, mean_e13, mean_e14, mean_e15, mean_e16, mean_e17]
mean_eop_obj2 = [mean_e6_op, mean_e7_op, mean_e8_op, mean_e9_op, mean_e10_op, mean_e11_op, mean_e12_op, mean_e13_op, mean_e14_op, mean_e15_op, mean_e16_op, mean_e17_op]
stdev_e_obj2 = [stdev_e6, stdev_e7, stdev_e8, stdev_e9, stdev_e10, stdev_e11, stdev_e12, stdev_e13, stdev_e14, stdev_e15, stdev_e16, stdev_e17]
stdev_eop_obj2 = [stdev_e6_op, stdev_e7_op, stdev_e8_op, stdev_e9_op, stdev_e10_op, stdev_e11_op, stdev_e12_op, stdev_e13_op, stdev_e14_op, stdev_e15_op, stdev_e16_op, stdev_e17_op]

In [ ]:
## Objective 2 - plot calculate errors


# Calculate the x-axis positions for each set of bars
x = np.arange(len(mean_e_obj2))

fig, ax = plt.subplots()

fig.set_size_inches(15*cm, 8*cm)


# Plot the bars side by side
ax.bar(x - 0.2, mean_e_obj2, yerr=stdev_e_obj2, width=0.4, color='tab:blue', align='center', label='estimated')
ax.bar(x + 0.2, mean_eop_obj2, yerr=stdev_eop_obj2, width=0.4, color='orange', align='center', label='optimised')

ax.set_yscale('log')
ax.set_ylabel('Mean relative error squared')

custom_labels = ['Error 6', 'Error 7', 'Error 8', 'Error 9', 'Error 10','Error 11', 'Error 12', 'Error 13', 'Error 14','Error 15', 'Error 16', 'Error 17']
ax.set_xticks(x)
ax.set_xticklabels(custom_labels,rotation=45)
ax.tick_params(axis='x', which='both', bottom=False, top=False)

#ax.legend(['estimated', 'optimised'], loc='upper left', bbox_to_anchor=(1.0, 1.0))
ax.legend(loc='best', prop={'size': 9})

modplot.set_axes_properties(ax)
plt.tick_params(bottom = False) 


In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(5*cm, 8*cm)

ax.bar(['Estimated', 'Optimised'], [mean_tot2, mean_tot2_op], yerr = [stdev_tot2, stdev_tot2_op], color=['tab:blue', 'orange'])
#ax.set_ylabel('Sum of mean relative errors squared')
ax.set_ylabel('Mean total relative error squared')
ax.tick_params(axis='x', which=u'both',length=0) # hide x axis ticks 
#plt.yscale('log')
modplot.set_axes_properties(ax)
plt.tick_params(bottom = False) 


In [ ]:
# Objective 2 - plot calculate errors (single k)
## Objective 1 - calculate errors (and variability) of non-optimsed vs optimised parameters

#Non optimised paramters 
e6_list_s = []
e7_list_s = []
e8_list_s = []
e9_list_s = []
e10_list_s = []
e11_list_s = []
e12_list_s = []
e13_list_s = []
e14_list_s = []
e15_list_s = []
e16_list_s = []
e17_list_s = []
tot2_list_s = []

for i in range(100):
    e6, e7, e8, e9, e10, e11, e12, e13, e14, e15, e16, e17, toterrs2 = objective2_test(obj1_fourteen_param)
    e6_list_s.append(e6)
    e7_list_s.append(e7)
    e8_list_s.append(e8)
    e9_list_s.append(e9)
    e10_list_s.append(e10)
    e11_list_s.append(e11)
    e12_list_s.append(e12)
    e13_list_s.append(e13)
    e14_list_s.append(e14)
    e15_list_s.append(e15)
    e16_list_s.append(e16)
    e17_list_s.append(e17)
    tot2_list_s.append(toterrs2)


mean_e6_s = mean(e6_list_s)
mean_e7_s = mean(e7_list_s)
mean_e8_s = mean(e8_list_s)
mean_e9_s = mean(e9_list_s)
mean_e10_s = mean(e10_list_s)
mean_e11_s = mean(e11_list_s)
mean_e12_s = mean(e12_list_s)
mean_e13_s = mean(e13_list_s)
mean_e14_s = mean(e14_list_s)
mean_e15_s = mean(e15_list_s)
mean_e16_s = mean(e16_list_s)
mean_e17_s = mean(e17_list_s)
mean_tot2_s = mean(tot2_list_s)

stdev_e6_s = np.std(e6_list_s)
stdev_e7_s = np.std(e7_list_s)
stdev_e8_s = np.std(e8_list_s)
stdev_e9_s = np.std(e9_list_s)
stdev_e10_s = np.std(e10_list_s)
stdev_e11_s = np.std(e11_list_s)
stdev_e12_s = np.std(e12_list_s)
stdev_e13_s = np.std(e13_list_s)
stdev_e14_s = np.std(e14_list_s)
stdev_e15_s = np.std(e15_list_s)
stdev_e16_s = np.std(e16_list_s)
stdev_e17_s = np.std(e17_list_s)
stdev_tot2_s = np.std(tot2_list_s)


#Optimised parameters 
e6_list_op_s = []
e7_list_op_s= []
e8_list_op_s = []
e9_list_op_s = []
e10_list_op_s = []
e11_list_op_s = []
e12_list_op_s = []
e13_list_op_s = []
e14_list_op_s = []
e15_list_op_s = []
e16_list_op_s = []
e17_list_op_s = []
tot2_list_op_s = []

for i in range(100):
    e6_op, e7_op, e8_op, e9_op, e10_op, e11_op, e12_op, e13_op, e14_op, e15_op, e16_op, e17_op, toterrs2_op = objective2_test(obj2_single_param)#
    e6_list_op_s.append(e6_op)
    e7_list_op_s.append(e7_op)
    e8_list_op_s.append(e8_op)
    e9_list_op_s.append(e9_op)
    e10_list_op_s.append(e10_op)
    e11_list_op_s.append(e11_op)
    e12_list_op_s.append(e12_op)
    e13_list_op_s.append(e13_op)
    e14_list_op_s.append(e14_op)
    e15_list_op_s.append(e15_op)
    e16_list_op_s.append(e16_op)
    e17_list_op_s.append(e17_op)
    tot2_list_op_s.append(toterrs2_op)


mean_e6_op_s = mean(e6_list_op_s)
mean_e7_op_s = mean(e7_list_op_s)
mean_e8_op_s = mean(e8_list_op_s)
mean_e9_op_s = mean(e9_list_op_s)
mean_e10_op_s = mean(e10_list_op_s)
mean_e11_op_s = mean(e11_list_op_s)
mean_e12_op_s = mean(e12_list_op_s)
mean_e13_op_s = mean(e13_list_op_s)
mean_e14_op_s = mean(e14_list_op_s)
mean_e15_op_s = mean(e15_list_op_s)
mean_e16_op_s = mean(e16_list_op_s)
mean_e17_op_s = mean(e17_list_op_s)
mean_tot2_op_s = mean(tot2_list_op_s)

stdev_e6_op_s = np.std(e6_list_op_s)
stdev_e7_op_s = np.std(e7_list_op_s)
stdev_e8_op_s = np.std(e8_list_op_s)
stdev_e9_op_s = np.std(e9_list_op_s)
stdev_e10_op_s = np.std(e10_list_op_s)
stdev_e11_op_s = np.std(e11_list_op_s)
stdev_e12_op_s = np.std(e12_list_op_s)
stdev_e13_op_s = np.std(e13_list_op_s)
stdev_e14_op_s = np.std(e14_list_op_s)
stdev_e15_op_s = np.std(e15_list_op_s)
stdev_e16_op_s = np.std(e16_list_op_s)
stdev_e17_op_s = np.std(e17_list_op_s)
stdev_tot2_op_s = np.std(tot2_list_op_s)

In [ ]:
#Put into lists
mean_e_obj2_s =  [mean_e6_s, mean_e7_s, mean_e8_s, mean_e9_s, mean_e10_s, mean_e11_s, mean_e12_s, mean_e13_s, mean_e14_s, mean_e15_s, mean_e16_s, mean_e17_s]
mean_eop_obj2_s = [mean_e6_op_s, mean_e7_op_s, mean_e8_op_s, mean_e9_op_s, mean_e10_op_s, mean_e11_op_s, mean_e12_op_s, mean_e13_op_s, mean_e14_op_s, mean_e15_op_s, mean_e16_op_s, mean_e17_op_s]
stdev_e_obj2_s = [stdev_e6_s, stdev_e7_s, stdev_e8_s, stdev_e9_s, stdev_e10_s, stdev_e11_s, stdev_e12_s, stdev_e13_s, stdev_e14_s, stdev_e15_s, stdev_e16_s, stdev_e17_s]
stdev_eop_obj2_s = [stdev_e6_op_s, stdev_e7_op_s, stdev_e8_op_s, stdev_e9_op_s, stdev_e10_op_s, stdev_e11_op_s, stdev_e12_op_s, stdev_e13_op_s, stdev_e14_op_s, stdev_e15_op_s, stdev_e16_op_s, stdev_e17_op_s]

In [ ]:
## Objective 2 single - plot calculate errors


# Calculate the x-axis positions for each set of bars
x = np.arange(len(mean_e_obj2_s))

fig, ax = plt.subplots()
fig.set_size_inches(15*cm, 8*cm)


# Plot the bars side by side
ax.bar(x - 0.2, mean_e_obj2_s, yerr=stdev_e_obj2_s, width=0.4, color='tab:blue', align='center', label='estimated')
ax.bar(x + 0.2, mean_eop_obj2_s, yerr=stdev_eop_obj2_s, width=0.4, color='orange', align='center', label='optimised')

ax.set_yscale('log')
ax.set_ylabel('Mean relative error squared')

custom_labels = ['Error 6', 'Error 7', 'Error 8', 'Error 9', 'Error 10','Error 11', 'Error 12', 'Error 13', 'Error 14','Error 15', 'Error 16', 'Error 17']
ax.set_xticks(x)
ax.set_xticklabels(custom_labels, rotation=45)
ax.tick_params(axis='x', which='both', bottom=False, top=False)

#ax.legend(loc='lower center', bbox_to_anchor=(0.5, -0.45), ncol=2, prop={'size': 9})
ax.legend(loc='best', prop={'size': 9})

modplot.set_axes_properties(ax)
plt.tick_params(bottom = False) 


In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(5*cm, 8*cm)

ax.bar(['Estimated', 'Optimised'], [mean_tot2_s, mean_tot2_op_s], yerr = [stdev_tot2_s, stdev_tot2_op_s], color=['tab:blue', 'orange'])
#ax.set_ylabel('Sum of mean relative errors squared')
ax.set_ylabel('Mean total relative error squared')

ax.tick_params(axis='x', which=u'both',length=0) # hide x axis ticks 
#plt.yscale('log')

modplot.set_axes_properties(ax)
plt.tick_params(bottom = False) 


In [ ]:
## Objective 2 single - plot calculate errors


# Calculate the x-axis positions for each set of bars
x = np.arange(len(mean_e_obj2_s))

fig, ax = plt.subplots()
fig.set_size_inches(17*cm, 8*cm)


# Plot the bars side by side
ax.bar(x - 0.28, mean_e_obj2_s, yerr=stdev_e_obj2_s, width=0.24, color='tab:blue', align='center', label='estimated')
ax.bar(x + 0, mean_eop_obj2, yerr=stdev_eop_obj2, width=0.24, color='orange', align='center', label='optimised, multiple k')
ax.bar(x + 0.28, mean_eop_obj2_s, yerr=stdev_eop_obj2_s, width=0.24, color='tab:red', align='center', label='optimised, global k')


ax.set_yscale('log')
ax.set_ylabel('Mean relative error squared')

custom_labels = ['Error 6', 'Error 7', 'Error 8', 'Error 9', 'Error 10','Error 11', 'Error 12', 'Error 13', 'Error 14','Error 15', 'Error 16', 'Error 17']
ax.set_xticks(x)
ax.set_xticklabels(custom_labels, rotation=45)
ax.tick_params(axis='x', which='both', bottom=False, top=False)

#ax.legend(loc='lower center', bbox_to_anchor=(0.5, -0.45), ncol=2, prop={'size': 9})
ax.legend(loc='best', prop={'size': 9})

modplot.set_axes_properties(ax)
plt.tick_params(bottom = False) 


In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(7.5*cm, 8*cm)

ax.bar(['Estimated', 'Multiple k', 'Gobal k'], [mean_tot2_s, mean_tot2_op_s, mean_tot2_op_s], yerr = [stdev_tot2_s, stdev_tot2_op, stdev_tot2_op_s], color=['tab:blue', 'orange', 'tab:red'])
#ax.set_ylabel('Sum of mean relative errors squared')
ax.set_ylabel('Mean total relative error squared')

ax.tick_params(axis='x', which=u'both',length=0) # hide x axis ticks 
#plt.yscale('log')

modplot.set_axes_properties(ax)
plt.tick_params(bottom = False) 


## Plotting output stats comparisonns (data & sim)

In [ ]:
dictsimstats = stats_optioutput(finalparameters) # dicrionary of stats outputs


In [ ]:
#putting stats into lists for plotting

simstats = [dictsimstats['stat1'], dictsimstats['stat2'],dictsimstats['stat3'],dictsimstats['stat4'],dictsimstats['stat5'],dictsimstats['stat6'],dictsimstats['stat7'], dictsimstats['stat8'], dictsimstats['stat9'], dictsimstats['stat10'],dictsimstats['stat11'],
            dictsimstats['stat12'],dictsimstats['stat13'],dictsimstats['stat14'],dictsimstats['stat15'],dictsimstats['stat16'],
            dictsimstats['stat17']]

datastats = [Dstat1, Dstat2, Dstat3, Dstat4, Dstat5, Dstat6, Dstat7, Dstat8, Dstat9, Dstat10, Dstat11, Dstat12, Dstat13, Dstat14, Dstat15, Dstat16, Dstat17]

In [ ]:
dataerr = [Derr1,Derr2,Derr3,Derr4,Derr5,Derr6,Derr7,Derr8,Derr9,Derr10,Derr11,Derr12,Derr13,Derr14,Derr15,Derr16,Derr17]

In [ ]:
subplot_titles = ['P immobile->mobile','P mobile -> immobile', 'mean immobile duration (sec)', 'stdev immobile duration (sec)', 'fraction always immobile',
                  'P forward->backward', 'P backward->forward', 'P forward->forward', 'P backward->backward', 'P immobile->backward',
                  'P immobile->forward', 'P backward->immobile', 'P forward->immobile', 'mean forward run duration (sec)','stdev forward run duration (sec)',
                  'mean backward run duration (sec)', 'stdev backward run duration (sec)']

In [ ]:
'''
lower_bounds_CI = [interval[0][0] for interval in dataerr]
upper_bounds_CI = [interval[1][0] for interval in dataerr]
'''

In [ ]:
import textwrap
from matplotlib.lines import Line2D


In [ ]:
## OBJECTIVE 1 # not flipped


fig =plt.figure(figsize=(10, 5))
spec = mpl.gridspec.GridSpec(ncols=3, nrows=2)

cm = 1/2.54    
fig.set_size_inches(15*cm, 9*cm)

ax1 = fig.add_subplot(spec[0,0])
ax2 = fig.add_subplot(spec[0,1])
ax3 = fig.add_subplot(spec[0,2])
ax4 = fig.add_subplot(spec[1,0])
ax5 = fig.add_subplot(spec[1,1])



for i in range(len(fig.axes)):
    ax = fig.axes[i]
    interval = dataerr[i]
  
    x = ['data', 'sim']
    y = [datastats[i], simstats[i]]
    lower_bound = interval[0][0]
    upper_bound = interval[1][0]
    #error = (np.abs([(y - lower_bound), (upper_bound - y)]))
    error = (np.abs([y - np.array([lower_bound,y[1]]), (np.array([lower_bound,y[1]]) - y)]))

    ax.bar(x, y, color=['grey', 'k'])
    ax.errorbar(x, y, yerr=error, fmt='none', ecolor='k', capsize=4)
    
    ax.xaxis.set_visible(False)
    

    ax.set_title(subplot_titles[i])
    wrapped_title = textwrap.fill(subplot_titles[i], 20)  # Adjust the width as desired
    ax.set_title(wrapped_title)

plt.subplots_adjust(hspace=0.5, wspace=0.3)


fig.legend(
    handles=[
        Line2D([], [], c='grey', label="Data",linewidth=5),
        Line2D([], [], c='k', label="Simulation",linewidth=5)
    ],
    loc="lower center", bbox_to_anchor=(0.5, -0.05), ncol=2,
    prop={'size': 9}
)

for axis in (ax1,ax2,ax3,ax4,ax5):
    modplot.set_axes_properties(axis)


fig.tight_layout() 

In [ ]:
## OBJECTIVE 2

fig =plt.figure(figsize=(10, 10))
spec = mpl.gridspec.GridSpec(ncols=3, nrows=4)

cm = 1/2.54    
fig.set_size_inches(15*cm, 17*cm)

ax1 = fig.add_subplot(spec[0,0])
ax2 = fig.add_subplot(spec[0,1])
ax3 = fig.add_subplot(spec[0,2])
ax4 = fig.add_subplot(spec[1,0])
ax5 = fig.add_subplot(spec[1,1])
ax6 = fig.add_subplot(spec[1,2])
ax7 = fig.add_subplot(spec[2,0])
ax8 = fig.add_subplot(spec[2,1])
ax9 = fig.add_subplot(spec[2,2])
ax10 = fig.add_subplot(spec[3,0])
ax11 = fig.add_subplot(spec[3,1])
ax12 = fig.add_subplot(spec[3,2])


for i in range(len(fig.axes)):
    ax = fig.axes[i]
    interval = dataerr[i+5]
  
    x = ['data', 'sim']
    y = [datastats[i+5], simstats[i+5]]
    lower_bound = interval[0][0]
    upper_bound = interval[1][0]
    ###error = (np.abs([(y - lower_bound), (upper_bound - y)]))
    error = (np.abs([y - np.array([lower_bound,y[1]]), (np.array([lower_bound,y[1]]) - y)]))
    
    ax.bar(x, y, color=['grey', 'k'])
    ax.errorbar(x, y, yerr=error, fmt='none', ecolor='k', capsize=4)
    
    ax.xaxis.set_visible(False)
    
    ax.set_title(subplot_titles[i+5])
    wrapped_title = textwrap.fill(subplot_titles[i+5], 20)  # Adjust the width as desired
    ax.set_title(wrapped_title)
    
    #sim_handle = Line2D([], [], color='k', marker='s', markersize=8, linestyle='-', label='Sim')
    #handles.append(sim_handle)

# add legend
#fig.legend(['Data', 'Sim'], loc='center right', bbox_to_anchor=(1.1, 0.5))

#fig.legend(handles=handles, labels=['Data', 'Sim'], loc='center right', bbox_to_anchor=(1.2, 0.5))

plt.subplots_adjust(hspace=0.5, wspace=0.3)

fig.legend(
    handles=[
        Line2D([], [], c='grey', label="Data",linewidth=5),
        Line2D([], [], c='k', label="Sim",linewidth=5)
    ],
    loc="lower center", bbox_to_anchor=(0.5, -0.05), ncol=3,
    prop={'size': 9}
)


for axis in (ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8,ax9,ax10,ax11,ax12):
    modplot.set_axes_properties(axis)

fig.tight_layout()

In [ ]:
dictsimstats_s = stats_optioutput(obj2_single_param) # dicrionary of stats outputs


In [ ]:
#putting stats into lists for plotting

simstats_s = [dictsimstats_s['stat1'], dictsimstats_s['stat2'],dictsimstats_s['stat3'],dictsimstats_s['stat4'],dictsimstats_s['stat5'],dictsimstats_s['stat6'],dictsimstats_s['stat7'], dictsimstats_s['stat8'], dictsimstats_s['stat9'], dictsimstats_s['stat10'],dictsimstats_s['stat11'],
            dictsimstats_s['stat12'],dictsimstats_s['stat13'],dictsimstats_s['stat14'],dictsimstats_s['stat15'],dictsimstats_s['stat16'],
            dictsimstats_s['stat17']]

datastats = [Dstat1, Dstat2, Dstat3, Dstat4, Dstat5, Dstat6, Dstat7, Dstat8, Dstat9, Dstat10, Dstat11, Dstat12, Dstat13, Dstat14, Dstat15, Dstat16, Dstat17]

In [ ]:
dataerr = [Derr1,Derr2,Derr3,Derr4,Derr5,Derr6,Derr7,Derr8,Derr9,Derr10,Derr11,Derr12,Derr13,Derr14,Derr15,Derr16,Derr17]

In [ ]:
subplot_titles = ['P immobile->mobile','P mobile -> immobile', 'mean immobile duration (sec)', 'stdev immobile duration (sec)', 'fraction always immobile',
                  'P forward->backward', 'P backward->forward', 'P forward->forward', 'P backward->backward', 'P immobile->backward',
                  'P immobile->forward', 'P backward->immobile', 'P forward->immobile', 'mean forward run duration (sec)','stdev forward run duration (sec)',
                  'mean backward run duration (sec)', 'stdev backward run duration (sec)']

In [ ]:
## OBJECTIVE 2
from matplotlib.lines import Line2D

fig =plt.figure(figsize=(10, 10))
spec = mpl.gridspec.GridSpec(ncols=3, nrows=4)

cm = 1/2.54    
fig.set_size_inches(15*cm, 17*cm)

ax1 = fig.add_subplot(spec[0,0])
ax2 = fig.add_subplot(spec[0,1])
ax3 = fig.add_subplot(spec[0,2])
ax4 = fig.add_subplot(spec[1,0])
ax5 = fig.add_subplot(spec[1,1])
ax6 = fig.add_subplot(spec[1,2])
ax7 = fig.add_subplot(spec[2,0])
ax8 = fig.add_subplot(spec[2,1])
ax9 = fig.add_subplot(spec[2,2])
ax10 = fig.add_subplot(spec[3,0])
ax11 = fig.add_subplot(spec[3,1])
ax12 = fig.add_subplot(spec[3,2])


for i in range(len(fig.axes)):
    ax = fig.axes[i]
    interval = dataerr[i+5]
  
    x = ['data', 'sim']
    y = [datastats[i+5], simstats_s[i+5]]
    lower_bound = interval[0][0]
    upper_bound = interval[1][0]
    #error = (np.abs([(y - lower_bound), (upper_bound - y)]))
    error = (np.abs([y - np.array([lower_bound,y[1]]), (np.array([lower_bound,y[1]]) - y)]))

    ax.bar(x, y, color=['grey', 'k'])
    ax.errorbar(x, y, yerr=error, fmt='none', ecolor='k', capsize=4)
    
    ax.xaxis.set_visible(False)
    
    ax.set_title(subplot_titles[i+5])
    wrapped_title = textwrap.fill(subplot_titles[i+5], 20)  # Adjust the width as desired
    ax.set_title(wrapped_title)


plt.subplots_adjust(hspace=0.5, wspace=0.3)

fig.legend(
    handles=[
        Line2D([], [], c='grey', label="Data",linewidth=5),
        Line2D([], [], c='k', label="Sim",linewidth=5)
    ],
    loc="lower center", bbox_to_anchor=(0.5, -0.05), ncol=3,
    prop={'size': 9}
)


for axis in (ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8,ax9,ax10,ax11,ax12):
    modplot.set_axes_properties(axis)


fig.tight_layout() 

In [ ]:
## OBJECTIVE 2
from matplotlib.lines import Line2D

fig =plt.figure(figsize=(10, 10))
spec = mpl.gridspec.GridSpec(ncols=3, nrows=4)

cm = 1/2.54    
fig.set_size_inches(15*cm, 17*cm)

ax1 = fig.add_subplot(spec[0,0])
ax2 = fig.add_subplot(spec[0,1])
ax3 = fig.add_subplot(spec[0,2])
ax4 = fig.add_subplot(spec[1,0])
ax5 = fig.add_subplot(spec[1,1])
ax6 = fig.add_subplot(spec[1,2])
ax7 = fig.add_subplot(spec[2,0])
ax8 = fig.add_subplot(spec[2,1])
ax9 = fig.add_subplot(spec[2,2])
ax10 = fig.add_subplot(spec[3,0])
ax11 = fig.add_subplot(spec[3,1])
ax12 = fig.add_subplot(spec[3,2])


for i in range(len(fig.axes)):
    ax = fig.axes[i]
    interval = dataerr[i+5]
  
    x = ['data', 'sim multiple', 'sim global']
    y = [datastats[i+5], simstats[i+5], simstats_s[i+5]]
    lower_bound = interval[0][0]
    upper_bound = interval[1][0]
    #error = (np.abs([(y - lower_bound), (upper_bound - y)]))
    error = (np.abs([y - np.array([lower_bound,y[1], y[2]]), (np.array([lower_bound,y[1], y[2]]) - y)]))

    ax.bar(x, y, color=['grey', 'k', 'tab:red'])
    ax.errorbar(x, y, yerr=error, fmt='none', ecolor='k', capsize=4)
    
    ax.xaxis.set_visible(False)
    
    ax.set_title(subplot_titles[i+5])
    wrapped_title = textwrap.fill(subplot_titles[i+5], 20)  # Adjust the width as desired
    ax.set_title(wrapped_title)


plt.subplots_adjust(hspace=0.5, wspace=0.3)

fig.legend(
    handles=[
        Line2D([], [], c='grey', label="Data",linewidth=5),
        Line2D([], [], c='k', label="Simulation, multiple k",linewidth=5),
        Line2D([], [], c='tab:red', label="Simulation, global k",linewidth=5)
    ],
    loc="lower center", bbox_to_anchor=(0.5, -0.05), ncol=3,
    prop={'size': 9}
)


for axis in (ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8,ax9,ax10,ax11,ax12):
    modplot.set_axes_properties(axis)


fig.tight_layout() 

## A/R PROOF OF CONCEPT: Plotting output stats comparisonns (data & sim) 

In [ ]:
#input result (xbest) from 1st optimisation (seven parameters - mobility)#
obj1best = [8.22460486e-01, 2.91345744e-06, 1.79508367e-03, 5.03262508e+00, 2.36216914e-04, 1.47111305e+00, 9.48665919e-01] #v4.1
# scaled result seven parameters (to be between 0 and 1)
result_param_seven_opti_scaled = scale_seven_parameters(obj1best)

#inpput result (xbest) from 2nd optimisation (four parameters - antero/retro)#
obj2single1 = [0.5, 0.5, 0.5, 0.5] 
obj2single2 = [0.2, 0.2, 0.2, 0.2]
obj2single3 = [0.8, 0.8, 0.8, 0.8]

##parameters after Obj_2 multiple optimisation
finalparameters1 = weighed_parameters_multiple(result_param_seven_opti_scaled, obj2single1, scale=False) 
finalparameters2 = weighed_parameters_multiple(result_param_seven_opti_scaled, obj2single2, scale=False)
finalparameters3 = weighed_parameters_multiple(result_param_seven_opti_scaled, obj2single3, scale=False) 

In [ ]:
#simulation stats outputs
dictsimstats1,bw1, fw1, bwtrack1, fwtrack1, bwnum1, fwnum1, bwlmean1, fwlmean1 = stats_optioutput_plus(finalparameters1) # dictionary of stats outputs
dictsimstats2,bw2, fw2, bwtrack2, fwtrack2, bwnum2, fwnum2, bwlmean2, fwlmean2 = stats_optioutput_plus(finalparameters2) # dictionary of stats outputs
dictsimstats3,bw3, fw3, bwtrack3, fwtrack3, bwnum3, fwnum3, bwlmean3, fwlmean3 = stats_optioutput_plus(finalparameters3) # dictionary of stats outputs




In [ ]:
obj2single4 = [0.1, 0.1, 0.1, 0.1]
obj2single5 = [0.9, 0.9, 0.9, 0.9]

finalparameters4 = weighed_parameters_multiple(result_param_seven_opti_scaled, obj2single4, scale=False)
finalparameters5 = weighed_parameters_multiple(result_param_seven_opti_scaled, obj2single5, scale=False) 

dictsimstats4,bw4, fw4, bwtrack4, fwtrack4, bwnum4, fwnum4, bwlmean4, fwlmean4 = stats_optioutput_plus(finalparameters4) # dictionary of stats outputs
dictsimstats5,bw5, fw5, bwtrack5, fwtrack5, bwnum5, fwnum5, bwlmean5, fwlmean5 = stats_optioutput_plus(finalparameters5) # dictionary of stats outputs



In [ ]:
dataerr = [Derr1,Derr2,Derr3,Derr4,Derr5,Derr6,Derr7,Derr8,Derr9,Derr10,Derr11,Derr12,Derr13,Derr14,Derr15,Derr16,Derr17]
datastats = [Dstat1, Dstat2, Dstat3, Dstat4, Dstat5, Dstat6, Dstat7, Dstat8, Dstat9, Dstat10, Dstat11, Dstat12, Dstat13, Dstat14, Dstat15, Dstat16, Dstat17]

In [ ]:
subplot_titles = ['P immobile->mobile','P mobile -> immobile', 'mean immobile duration (sec)', 'stdev immobile duration (sec)', 'fraction always immobile',
                  'P forward->backward', 'P backward->forward', 'P forward->forward', 'P backward->backward', 'P immobile->backward',
                  'P immobile->forward', 'P backward->immobile', 'P forward->immobile', 'mean forward run duration (sec)','stdev forward run duration (sec)',
                  'mean backward run duration (sec)', 'stdev backward run duration (sec)']

In [ ]:

simstats1 = [dictsimstats1['stat1'], dictsimstats1['stat2'],dictsimstats1['stat3'],dictsimstats1['stat4'],dictsimstats1['stat5'],dictsimstats1['stat6'],dictsimstats1['stat7'], dictsimstats1['stat8'], dictsimstats1['stat9'], dictsimstats1['stat10'],dictsimstats1['stat11'],
            dictsimstats1['stat12'],dictsimstats1['stat13'],dictsimstats1['stat14'],dictsimstats1['stat15'],dictsimstats1['stat16'],
            dictsimstats1['stat17']]


simstats2 = [dictsimstats2['stat1'], dictsimstats2['stat2'],dictsimstats2['stat3'],dictsimstats2['stat4'],dictsimstats2['stat5'],dictsimstats2['stat6'],dictsimstats2['stat7'], dictsimstats2['stat8'], dictsimstats2['stat9'], dictsimstats2['stat10'],dictsimstats2['stat11'],
            dictsimstats2['stat12'],dictsimstats2['stat13'],dictsimstats2['stat14'],dictsimstats2['stat15'],dictsimstats2['stat16'],
            dictsimstats2['stat17']]

simstats3 = [dictsimstats3['stat1'], dictsimstats3['stat2'],dictsimstats3['stat3'],dictsimstats3['stat4'],dictsimstats3['stat5'],dictsimstats3['stat6'],dictsimstats3['stat7'], dictsimstats3['stat8'], dictsimstats3['stat9'], dictsimstats3['stat10'],dictsimstats3['stat11'],
            dictsimstats3['stat12'],dictsimstats3['stat13'],dictsimstats3['stat14'],dictsimstats3['stat15'],dictsimstats3['stat16'],
            dictsimstats3['stat17']]

In [ ]:
simstats4 = [dictsimstats4['stat1'], dictsimstats4['stat2'],dictsimstats4['stat3'],dictsimstats4['stat4'],dictsimstats4['stat5'],dictsimstats3['stat6'],dictsimstats4['stat7'], dictsimstats4['stat8'], dictsimstats4['stat9'], dictsimstats4['stat10'],dictsimstats4['stat11'],
            dictsimstats4['stat12'],dictsimstats4['stat13'],dictsimstats4['stat14'],dictsimstats4['stat15'],dictsimstats4['stat16'],
            dictsimstats4['stat17']]

simstats5 = [dictsimstats5['stat1'], dictsimstats5['stat2'],dictsimstats5['stat3'],dictsimstats5['stat4'],dictsimstats5['stat5'],dictsimstats5['stat6'],dictsimstats5['stat7'], dictsimstats5['stat8'], dictsimstats5['stat9'], dictsimstats5['stat10'],dictsimstats5['stat11'],
            dictsimstats5['stat12'],dictsimstats5['stat13'],dictsimstats5['stat14'],dictsimstats5['stat15'],dictsimstats5['stat16'],
            dictsimstats5['stat17']]

In [ ]:
## OBJECTIVE 2
from matplotlib.lines import Line2D

fig =plt.figure(figsize=(10, 10))
spec = mpl.gridspec.GridSpec(ncols=3, nrows=4)

ax1 = fig.add_subplot(spec[0,0])
ax2 = fig.add_subplot(spec[0,1])
ax3 = fig.add_subplot(spec[0,2])
ax4 = fig.add_subplot(spec[1,0])
ax5 = fig.add_subplot(spec[1,1])
ax6 = fig.add_subplot(spec[1,2])
ax7 = fig.add_subplot(spec[2,0])
ax8 = fig.add_subplot(spec[2,1])
ax9 = fig.add_subplot(spec[2,2])
ax10 = fig.add_subplot(spec[3,0])
ax11 = fig.add_subplot(spec[3,1])
ax12 = fig.add_subplot(spec[3,2])


cm = 1/2.54    
fig.set_size_inches(15*cm, 17*cm)


for i in range(len(fig.axes)):
    ax = fig.axes[i]
    interval = dataerr[i+5]
  
    x = ['sim1', 'sim2', 'sim3']
    y = [simstats1[i+5], simstats2[i+5], simstats3[i+5]]


    ax.bar(x, y, color=['tab:blue', 'orange', 'tab:red'])
    
    ax.xaxis.set_visible(False)
    
    ax.set_title(subplot_titles[i+5])
    wrapped_title = textwrap.fill(subplot_titles[i+5], 20)  # Adjust the width as desired
    ax.set_title(wrapped_title, fontsize=15)
    

plt.subplots_adjust(hspace=0.5, wspace=0.5)

fig.legend(
    handles=[
        Line2D([], [], c='tab:blue', label="k = 0.5",linewidth=5),
        Line2D([], [], c='orange', label="k = 0.2 (anterograde bias)",linewidth=5),
        Line2D([], [], c='tab:red', label="k = 0.8 (retrograde bias)",linewidth=5)
    ],
    loc="lower center", bbox_to_anchor=(0.51, -0.05), ncol=3,
    prop={'size': 9}

)

for axis in (ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8,ax9,ax10,ax11,ax12):
    modplot.set_axes_properties(axis)


fig.tight_layout() 


In [ ]:
## OBJECTIVE 2
from matplotlib.lines import Line2D

fig =plt.figure(figsize=(10, 10))
spec = mpl.gridspec.GridSpec(ncols=3, nrows=4)

ax1 = fig.add_subplot(spec[0,0])
ax2 = fig.add_subplot(spec[0,1])
ax3 = fig.add_subplot(spec[0,2])
ax4 = fig.add_subplot(spec[1,0])
ax5 = fig.add_subplot(spec[1,1])
ax6 = fig.add_subplot(spec[1,2])
ax7 = fig.add_subplot(spec[2,0])
ax8 = fig.add_subplot(spec[2,1])
ax9 = fig.add_subplot(spec[2,2])
ax10 = fig.add_subplot(spec[3,0])
ax11 = fig.add_subplot(spec[3,1])
ax12 = fig.add_subplot(spec[3,2])


cm = 1/2.54    
fig.set_size_inches(15*cm, 17*cm)


for i in range(len(fig.axes)):
    ax = fig.axes[i]
    interval = dataerr[i+5]
  
    x = ['sim1', 'sim4', 'sim5']
    y = [simstats1[i+5], simstats4[i+5], simstats5[i+5]]
    #lower_bound = interval[0][0]
    #upper_bound = interval[1][0]
    #error = (np.abs([y - np.array([lower_bound,y[1]]), (np.array([lower_bound,y[1]]) - y)]))

    ax.bar(x, y, color=['tab:blue', 'orange', 'tab:red'])
    #ax.errorbar(x, y, yerr=error, fmt='none', ecolor='k', capsize=4)
    
    ax.xaxis.set_visible(False)
    
    ax.set_title(subplot_titles[i+5])
    wrapped_title = textwrap.fill(subplot_titles[i+5], 20)  # Adjust the width as desired
    ax.set_title(wrapped_title, fontsize=15)
    

plt.subplots_adjust(hspace=0.5, wspace=0.5)

fig.legend(
    handles=[
        Line2D([], [], c='tab:blue', label="k = 0.5",linewidth=5),
        Line2D([], [], c='orange', label="k = 0.1 (anterograde bias)",linewidth=5),
        Line2D([], [], c='tab:red', label="k = 0.9 (retrograde bias)",linewidth=5)
    ],
    #loc='center right', bbox_to_anchor=(1.3, 0.5)
    loc="lower center", bbox_to_anchor=(0.51, -0.05), ncol=3,
    prop={'size': 9}

)

for axis in (ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8,ax9,ax10,ax11,ax12):
    modplot.set_axes_properties(axis)


fig.tight_layout() 


In [ ]:
## OBJECTIVE 1
from matplotlib.lines import Line2D

fig =plt.figure(figsize=(10, 5))
spec = mpl.gridspec.GridSpec(ncols=3, nrows=2)

ax1 = fig.add_subplot(spec[0,0])
ax2 = fig.add_subplot(spec[0,1])
ax3 = fig.add_subplot(spec[0,2])
ax4 = fig.add_subplot(spec[1,0])
ax5 = fig.add_subplot(spec[1,1])

cm = 1/2.54    
fig.set_size_inches(15*cm, 9*cm)

for i in range(len(fig.axes)):
    ax = fig.axes[i]
    #interval = dataerr[i]
  
    x = ['sim1', 'sim2', 'sim3']
    y = [simstats1[i], simstats2[i], simstats3[i]]


    ax.bar(x, y, color=['tab:blue', 'orange', 'tab:red'])
    
    ax.xaxis.set_visible(False)
    
    ax.set_title(subplot_titles[i], fontsize=15)
    wrapped_title = textwrap.fill(subplot_titles[i], 20)  # Adjust the width as desired
    ax.set_title(wrapped_title, fontsize=15)
    

plt.subplots_adjust(hspace=0.5, wspace=0.3)

fig.legend(
    handles=[
        Line2D([], [], c='tab:blue', label="k = 0.5",linewidth=5),
        Line2D([], [], c='orange', label="k = 0.2 (anterograde bias)",linewidth=5),
        Line2D([], [], c='tab:red', label="k = 0.8 (retrograde bias)",linewidth=5)
    ],
    loc="lower center", bbox_to_anchor=(0.51, -0.05), ncol=3,
    prop={'size': 9}
)

for axis in (ax1,ax2,ax3,ax4,ax5):
    modplot.set_axes_properties(axis)


fig.tight_layout() 